In [5]:
import torch
from monai.networks.nets import UNet
from monai.metrics import DiceMetric
from PIL import Image
import os
import matplotlib.pyplot as plt
from monai.networks.layers import Norm
import numpy as np
from monai.transforms import Transform
from monai.transforms import (
    Activations,
    EnsureChannelFirstd,
    Compose,
    ScaleIntensityd,
    ToTensor,
    AsDiscrete,
    Resized
)
from PIL import Image


# Load the pre-trained model
model_path = r"D:\Aytekin\CovidMasterThesis\models\Resize512_GeneralizedDiceLoss_epoch20_0_8679_valmeandice\512green_alldata_generalized_diceloss_epoch20_16_256_evenhigherlr.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(
    spatial_dims=2,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=3,
    norm = Norm.BATCH,
).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

# Define the transformations
test_transforms = Compose([
    EnsureChannelFirstd(keys=["img"], channel_dim = "no_channel"),
    ScaleIntensityd(keys=["img"]),
    Resized(keys=["img"], spatial_size=(512, 512), mode="nearest"),
    # Add any additional transformations if needed
])

# Define the transformations for segmentations
seg_transforms = Compose([
    EnsureChannelFirstd(keys=["img"], channel_dim="no_channel"),
    Resized(keys=["img"], spatial_size=(512, 512), mode="nearest"),
    # Add any additional transformations if needed
])


# Directory paths for input and segmentation images
input_image_dir = r"E:\Aytekin\Data\Input"
segmentation_dir = r"E:\Aytekin\Data\Segmentation"
ignore_dir = r"E:\Aytekin\Data\UniqueIntensityLessThan750"




# List of strings to exclude from image names
exclude_strings = ["12_0500", "29_0520", "01_0520", "08_0420", "29_0820", "29_1120", "29_1420", "29_1720", "29_2020", "29_2320"]

# List all .tif image files in the input and segmentation directories
input_image_files = [os.path.join(input_image_dir, filename) for filename in os.listdir(input_image_dir) if filename.endswith(".tif") and all(exclude not in filename for exclude in exclude_strings)]
segmentation_files = [os.path.join(segmentation_dir, filename) for filename in os.listdir(segmentation_dir) if filename.endswith(".tif") and all(exclude not in filename for exclude in exclude_strings)]


# Extract file names without extensions
input_image_names = [os.path.splitext(os.path.basename(filename))[0] for filename in input_image_files]
segmentation_names = [os.path.splitext(os.path.basename(filename))[0] for filename in segmentation_files]

# Find the common file names between input and segmentation
common_names = set(input_image_names) & set(segmentation_names)

# List all .tif image files in the directories again (overwrite previous lists)
input_image_files = [os.path.join(input_image_dir, filename) for filename in os.listdir(input_image_dir) if filename.endswith(".tif")]
segmentation_files = [os.path.join(segmentation_dir, filename) for filename in os.listdir(segmentation_dir) if filename.endswith(".tif")]

# Exclude images from UniqueIntensityLessThan750 directory
ignore_files = [os.path.join(ignore_dir, filename) for filename in os.listdir(ignore_dir) if filename.endswith(".tif")]
ignore_file_names = [os.path.splitext(os.path.basename(filename))[0] for filename in ignore_files]

# Filter input_image_files and segmentation_files based on common names and ignoring specific names
input_image_files = [filename for filename in input_image_files if os.path.splitext(os.path.basename(filename))[0] in common_names and os.path.splitext(os.path.basename(filename))[0] not in ignore_file_names]
segmentation_files = [filename for filename in segmentation_files if os.path.splitext(os.path.basename(filename))[0] in common_names and os.path.splitext(os.path.basename(filename))[0] not in ignore_file_names]


post_trans = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)

# Iterate through test images
for input_image_file, segmentation_file in zip(input_image_files, segmentation_files):
    input_image = Image.open(input_image_file)
    ground_truth_segmentation = Image.open(segmentation_file)
    
    # Apply transformations to the input image
    test_data = test_transforms({"img": np.array(input_image, dtype=np.float32)})
    test_input = torch.tensor(test_data["img"]).unsqueeze(0).to(device)

    # Perform prediction
    with torch.no_grad():
        predicted = model(test_input)
        predicted_binary = post_trans(predicted)
    
     # Save the prediction mask
    save_path = r"E:\Aytekin\PredictionMasksGreen"
    filename = os.path.splitext(os.path.basename(input_image_file))[0] + ".tif"
    # Convert the predicted binary mask to a PIL image
    predicted_mask_pil = Image.fromarray(np.squeeze(predicted_binary.cpu().numpy()).astype(np.uint8))
    #Save the PIL image as a TIFF file with .tif extension
    predicted_mask_pil.save(os.path.join(save_path, filename), format='TIFF')
    print(f"{filename} saved. No problem.")
    
    ground_truth_data = seg_transforms({"img": np.array(ground_truth_segmentation, dtype=np.float32)})
    ground_truth = torch.tensor(ground_truth_data["img"]).unsqueeze(0).to(device)
    
    threshold = 0  # Define your threshold value

    # Convert non-zero values to 1 based on the threshold
    ground_truth_binary = torch.where(ground_truth > threshold, torch.tensor(1).to(device), torch.tensor(0).to(device))

    # Calculate Dice score
    dice_metric(y_pred=predicted_binary, y=ground_truth_binary)

# Aggregate the final mean dice result
dice_score = dice_metric.aggregate().item()
print(f"Mean Dice Score for the prediction set: {dice_score}")

# Reset the status for the next evaluation round
dice_metric.reset()


C:\Users\ge59gor\AppData\Local\Temp\ipykernel_18088\2489135766.py:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_input = torch.tensor(test_data["img"]).unsqueeze(0).to(device)
C:\Users\ge59gor\AppData\Local\Temp\ipykernel_18088\2489135766.py:116: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ground_truth = torch.tensor(ground_truth_data["img"]).unsqueeze(0).to(device)


R_01_0507_939_C1-1-green.tif saved. No problem.
R_01_0507_939_F1-1-green.tif saved. No problem.
R_01_0507_939_G1-1-green.tif saved. No problem.
R_01_0507_939_H1-1-green.tif saved. No problem.
R_01_0507_941_G1-1-green.tif saved. No problem.
R_01_0507_941_H1-1-green.tif saved. No problem.
R_01_0507_941_H2-1-green.tif saved. No problem.
R_01_0807_939_C1-1-green.tif saved. No problem.
R_01_0807_939_F1-1-green.tif saved. No problem.
R_01_0807_939_G1-1-green.tif saved. No problem.
R_01_0807_939_G2-1-green.tif saved. No problem.
R_01_0807_939_H1-1-green.tif saved. No problem.
R_01_0807_939_H3-1-green.tif saved. No problem.
R_01_0807_940_F5-1-green.tif saved. No problem.
R_01_0807_941_E1-1-green.tif saved. No problem.
R_01_0807_941_G1-1-green.tif saved. No problem.
R_01_0807_941_H1-1-green.tif saved. No problem.
R_01_0807_941_H2-1-green.tif saved. No problem.
R_01_1051_938_A6-1-green.tif saved. No problem.
R_01_1051_938_B2-1-green.tif saved. No problem.
R_01_1051_938_B3-1-green.tif saved. No p

R_01_1407_939_G2-1-green.tif saved. No problem.
R_01_1407_939_G3-1-green.tif saved. No problem.
R_01_1407_939_G4-1-green.tif saved. No problem.
R_01_1407_939_G5-1-green.tif saved. No problem.
R_01_1407_939_G7-1-green.tif saved. No problem.
R_01_1407_939_H1-1-green.tif saved. No problem.
R_01_1407_939_H10-1-green.tif saved. No problem.
R_01_1407_939_H11-1-green.tif saved. No problem.
R_01_1407_939_H12-1-green.tif saved. No problem.
R_01_1407_939_H2-1-green.tif saved. No problem.
R_01_1407_939_H3-1-green.tif saved. No problem.
R_01_1407_939_H4-1-green.tif saved. No problem.
R_01_1407_939_H6-1-green.tif saved. No problem.
R_01_1407_939_H7-1-green.tif saved. No problem.
R_01_1407_939_H8-1-green.tif saved. No problem.
R_01_1407_940_A12-1-green.tif saved. No problem.
R_01_1407_940_A3-1-green.tif saved. No problem.
R_01_1407_940_A4-1-green.tif saved. No problem.
R_01_1407_940_A7-1-green.tif saved. No problem.
R_01_1407_940_B1-1-green.tif saved. No problem.
R_01_1407_940_B10-1-green.tif saved.

R_01_1651_938_D12-1-green.tif saved. No problem.
R_01_1651_938_D2-1-green.tif saved. No problem.
R_01_1651_938_D3-1-green.tif saved. No problem.
R_01_1651_938_D4-1-green.tif saved. No problem.
R_01_1651_938_D5-1-green.tif saved. No problem.
R_01_1651_938_D6-1-green.tif saved. No problem.
R_01_1651_938_D7-1-green.tif saved. No problem.
R_01_1651_938_D9-1-green.tif saved. No problem.
R_01_1651_938_E1-1-green.tif saved. No problem.
R_01_1651_938_E2-1-green.tif saved. No problem.
R_01_1651_938_E3-1-green.tif saved. No problem.
R_01_1651_938_H1-1-green.tif saved. No problem.
R_01_1651_938_H10-1-green.tif saved. No problem.
R_01_1651_938_H11-1-green.tif saved. No problem.
R_01_1651_938_H12-1-green.tif saved. No problem.
R_01_1651_938_H2-1-green.tif saved. No problem.
R_01_1651_938_H3-1-green.tif saved. No problem.
R_01_1651_938_H4-1-green.tif saved. No problem.
R_01_1651_938_H5-1-green.tif saved. No problem.
R_01_1651_938_H6-1-green.tif saved. No problem.
R_01_1651_938_H7-1-green.tif saved. 

R_01_1707_940_G1-1-green.tif saved. No problem.
R_01_1707_940_G10-1-green.tif saved. No problem.
R_01_1707_940_G11-1-green.tif saved. No problem.
R_01_1707_940_G12-1-green.tif saved. No problem.
R_01_1707_940_G2-1-green.tif saved. No problem.
R_01_1707_940_G3-1-green.tif saved. No problem.
R_01_1707_940_G4-1-green.tif saved. No problem.
R_01_1707_940_G5-1-green.tif saved. No problem.
R_01_1707_940_G6-1-green.tif saved. No problem.
R_01_1707_940_G7-1-green.tif saved. No problem.
R_01_1707_940_G8-1-green.tif saved. No problem.
R_01_1707_940_G9-1-green.tif saved. No problem.
R_01_1707_940_H1-1-green.tif saved. No problem.
R_01_1707_940_H10-1-green.tif saved. No problem.
R_01_1707_940_H11-1-green.tif saved. No problem.
R_01_1707_940_H12-1-green.tif saved. No problem.
R_01_1707_940_H2-1-green.tif saved. No problem.
R_01_1707_940_H3-1-green.tif saved. No problem.
R_01_1707_940_H4-1-green.tif saved. No problem.
R_01_1707_940_H5-1-green.tif saved. No problem.
R_01_1707_940_H6-1-green.tif saved

...

R_30_1120_935_F7-1-green.tif saved. No problem.
R_30_1120_935_F8-1-green.tif saved. No problem.
R_30_1120_935_F9-1-green.tif saved. No problem.
R_30_1120_935_G1-1-green.tif saved. No problem.
R_30_1120_935_G10-1-green.tif saved. No problem.
R_30_1120_935_G11-1-green.tif saved. No problem.
R_30_1120_935_G12-1-green.tif saved. No problem.
R_30_1120_935_G2-1-green.tif saved. No problem.
R_30_1120_935_G3-1-green.tif saved. No problem.
R_30_1120_935_G4-1-green.tif saved. No problem.
R_30_1120_935_G5-1-green.tif saved. No problem.
R_30_1120_935_G6-1-green.tif saved. No problem.
R_30_1120_935_G7-1-green.tif saved. No problem.
R_30_1120_935_G8-1-green.tif saved. No problem.
R_30_1120_935_G9-1-green.tif saved. No problem.
R_30_1120_935_H1-1-green.tif saved. No problem.
R_30_1120_935_H10-1-green.tif saved. No problem.
R_30_1120_935_H11-1-green.tif saved. No problem.
R_30_1120_935_H12-1-green.tif saved. No problem.
R_30_1120_935_H2-1-green.tif saved. No problem.
R_30_1120_935_H3-1-green.tif saved

R_30_1420_934_A1-1-green.tif saved. No problem.
R_30_1420_934_A10-1-green.tif saved. No problem.
R_30_1420_934_A11-1-green.tif saved. No problem.
R_30_1420_934_A12-1-green.tif saved. No problem.
R_30_1420_934_A2-1-green.tif saved. No problem.
R_30_1420_934_A3-1-green.tif saved. No problem.
R_30_1420_934_A4-1-green.tif saved. No problem.
R_30_1420_934_A5-1-green.tif saved. No problem.
R_30_1420_934_A6-1-green.tif saved. No problem.
R_30_1420_934_A7-1-green.tif saved. No problem.
R_30_1420_934_A8-1-green.tif saved. No problem.
R_30_1420_934_B1-1-green.tif saved. No problem.
R_30_1420_934_B10-1-green.tif saved. No problem.
R_30_1420_934_B11-1-green.tif saved. No problem.
R_30_1420_934_B12-1-green.tif saved. No problem.
R_30_1420_934_B2-1-green.tif saved. No problem.
R_30_1420_934_B3-1-green.tif saved. No problem.
R_30_1420_934_B4-1-green.tif saved. No problem.
R_30_1420_934_B5-1-green.tif saved. No problem.
R_30_1420_934_B6-1-green.tif saved. No problem.
R_30_1420_934_B7-1-green.tif saved

R_30_1420_935_G12-1-green.tif saved. No problem.
R_30_1420_935_G2-1-green.tif saved. No problem.
R_30_1420_935_G3-1-green.tif saved. No problem.
R_30_1420_935_G4-1-green.tif saved. No problem.
R_30_1420_935_G5-1-green.tif saved. No problem.
R_30_1420_935_G6-1-green.tif saved. No problem.
R_30_1420_935_G7-1-green.tif saved. No problem.
R_30_1420_935_G8-1-green.tif saved. No problem.
R_30_1420_935_G9-1-green.tif saved. No problem.
R_30_1420_935_H1-1-green.tif saved. No problem.
R_30_1420_935_H10-1-green.tif saved. No problem.
R_30_1420_935_H11-1-green.tif saved. No problem.
R_30_1420_935_H12-1-green.tif saved. No problem.
R_30_1420_935_H2-1-green.tif saved. No problem.
R_30_1420_935_H3-1-green.tif saved. No problem.
R_30_1420_935_H4-1-green.tif saved. No problem.
R_30_1420_935_H5-1-green.tif saved. No problem.
R_30_1420_935_H6-1-green.tif saved. No problem.
R_30_1420_935_H7-1-green.tif saved. No problem.
R_30_1420_935_H8-1-green.tif saved. No problem.
R_30_1420_935_H9-1-green.tif saved. 

R_30_1720_934_A2-1-green.tif saved. No problem.
R_30_1720_934_A3-1-green.tif saved. No problem.
R_30_1720_934_A4-1-green.tif saved. No problem.
R_30_1720_934_A5-1-green.tif saved. No problem.
R_30_1720_934_A6-1-green.tif saved. No problem.
R_30_1720_934_A7-1-green.tif saved. No problem.
R_30_1720_934_A8-1-green.tif saved. No problem.
R_30_1720_934_B1-1-green.tif saved. No problem.
R_30_1720_934_B10-1-green.tif saved. No problem.
R_30_1720_934_B11-1-green.tif saved. No problem.
R_30_1720_934_B12-1-green.tif saved. No problem.
R_30_1720_934_B2-1-green.tif saved. No problem.
R_30_1720_934_B3-1-green.tif saved. No problem.
R_30_1720_934_B4-1-green.tif saved. No problem.
R_30_1720_934_B5-1-green.tif saved. No problem.
R_30_1720_934_B6-1-green.tif saved. No problem.
R_30_1720_934_B7-1-green.tif saved. No problem.
R_30_1720_934_B8-1-green.tif saved. No problem.
R_30_1720_934_B9-1-green.tif saved. No problem.
R_30_1720_934_C1-1-green.tif saved. No problem.
R_30_1720_934_C10-1-green.tif saved. 

R_30_1720_935_G6-1-green.tif saved. No problem.
R_30_1720_935_G7-1-green.tif saved. No problem.
R_30_1720_935_G8-1-green.tif saved. No problem.
R_30_1720_935_G9-1-green.tif saved. No problem.
R_30_1720_935_H1-1-green.tif saved. No problem.
R_30_1720_935_H10-1-green.tif saved. No problem.
R_30_1720_935_H11-1-green.tif saved. No problem.
R_30_1720_935_H12-1-green.tif saved. No problem.
R_30_1720_935_H2-1-green.tif saved. No problem.
R_30_1720_935_H3-1-green.tif saved. No problem.
R_30_1720_935_H4-1-green.tif saved. No problem.
R_30_1720_935_H5-1-green.tif saved. No problem.
R_30_1720_935_H6-1-green.tif saved. No problem.
R_30_1720_935_H7-1-green.tif saved. No problem.
R_30_1720_935_H8-1-green.tif saved. No problem.
R_30_1720_935_H9-1-green.tif saved. No problem.
R_30_1720_936_A1-1-green.tif saved. No problem.
R_30_1720_936_A10-1-green.tif saved. No problem.
R_30_1720_936_A11-1-green.tif saved. No problem.
R_30_1720_936_A12-1-green.tif saved. No problem.
R_30_1720_936_A2-1-green.tif saved

In [6]:
print(len(segmentation_files))

10267


In [9]:
print(np.asarray(Image.open(r"E:\Aytekin\PredictionMasksGreen\R_01_0507_939_C1-1-green.tif")))

[[0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
